# Crash look at UK road traffic accident data

In [30]:
#Imports
from time import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px

from sklearn.neighbors import KDTree
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
from sklearn import metrics
# from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn

In [2]:
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Marat Sans']

In [3]:
######### Read up accidents
accidents_2018 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/dftRoadSafetyData_Accidents_2018.csv",low_memory=False)
accidents_2017 = pd.read_csv(r"http://data.dft.gov.uk.s3.amazonaws.com/road-accidents-safety-data/dftRoadSafetyData_Accidents_2017.zip",low_memory=False, compression='zip')
accidents_2016 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/dftRoadSafety_Accidents_2016.zip",low_memory=False, compression='zip')
accidents_2015 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/RoadSafetyData_Accidents_2015.zip",low_memory=False, compression='zip')
accidents_2014 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/DfTRoadSafety_Accidents_2014.zip",low_memory=False)

######### Read up casualties
casualties_2018 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/dftRoadSafetyData_Casualties_2018.csv",low_memory=False)
casualties_2017 = pd.read_csv(r"http://data.dft.gov.uk.s3.amazonaws.com/road-accidents-safety-data/dftRoadSafetyData_Casualties_2017.zip",low_memory=False, compression='zip')
casualties_2016 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/dftRoadSafetyData_Casualties_2016.zip",low_memory=False, compression='zip')
casualties_2015 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/RoadSafetyData_Casualties_2015.zip",low_memory=False, compression='zip')
casualties_2014 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/DfTRoadSafety_Casualties_2014.zip",low_memory=False, compression='zip')

######### Read up vehicles
vehicles_2018 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/dftRoadSafetyData_Vehicles_2018.csv",low_memory=False)
vehicles_2017 = pd.read_csv(r"http://data.dft.gov.uk.s3.amazonaws.com/road-accidents-safety-data/dftRoadSafetyData_Vehicles_2017.zip",low_memory=False, compression='zip')
vehicles_2016 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/dftRoadSafetyData_Vehicles_2016.zip",low_memory=False, compression='zip')
vehicles_2015 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/RoadSafetyData_Vehicles_2015.zip",low_memory=False, compression='zip')
vehicles_2014 = pd.read_csv(r"http://data.dft.gov.uk/road-accidents-safety-data/DfTRoadSafety_Vehicles_2014.zip",low_memory=False, compression='zip')

df_accidents = accidents_2018.append([accidents_2017, accidents_2016, accidents_2015, accidents_2014], sort=False)
df_vehicles = vehicles_2018.append([vehicles_2017, vehicles_2016, vehicles_2015, vehicles_2014], sort=False)
df_casualties = casualties_2018.append([casualties_2017, casualties_2016, casualties_2015, casualties_2014], sort=False)


In [4]:
df_accidents.dtypes

Accident_Index                                  object
Location_Easting_OSGR                          float64
Location_Northing_OSGR                         float64
Longitude                                      float64
Latitude                                       float64
Police_Force                                     int64
Accident_Severity                                int64
Number_of_Vehicles                               int64
Number_of_Casualties                             int64
Date                                            object
Day_of_Week                                      int64
Time                                            object
Local_Authority_(District)                       int64
Local_Authority_(Highway)                       object
1st_Road_Class                                   int64
1st_Road_Number                                  int64
Road_Type                                        int64
Speed_limit                                    float64
Junction_D

In [ ]:
df_casualties.dtypes
#pd.to_numeric(df_casualties['Accident_Index'],downcast='integer')

In [ ]:
df_vehicles.dtypes

In [ ]:
df_accidents["Time stamp"] = pd.DatetimeIndex(df_accidents["Date"] +" " + df_accidents["Time"],dayfirst=True, normalize=True)

df_accidents["Year"] = df_accidents["Time stamp"].dt.year
df_accidents["Month"] = df_accidents["Time stamp"].dt.month
df_accidents["Hour"] = df_accidents["Time stamp"].dt.hour

df_vehicles = df_vehicles.merge(df_accidents[["Accident_Index","Time stamp"]],left_on="Accident_Index",right_on="Accident_Index",how='left')
df_casualties = df_casualties.merge(df_accidents[["Accident_Index","Time stamp", "1st_Road_Class", "2nd_Road_Class","Location_Easting_OSGR", "Location_Northing_OSGR"]],
                                    left_on="Accident_Index",right_on="Accident_Index",how='left')

In [ ]:
years = df_accidents.groupby(["Year"]).size().keys()
accidents_by_year=df_accidents.groupby(["Year"]).size().values

fatalities_by_year=[]
for y in years:
    fatalities_by_year.append(df_casualties[(df_casualties["Time stamp"].dt.year == y) & (df_casualties["Casualty_Severity"]==1) 
                                          ].count()[0])

In [ ]:
fig, ax = plt.subplots()
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

ax.spines['bottom'].set_position(('data', 0))
#ax.spines['left'].set_position(('data', 0))

ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.tick_params(labelsize=14)
ax.grid(axis='x')

plt.plot(years, fatalities_by_year, color='#e91e63')
#plt.plot(years, serious_by_year, color='#e91e63')
#plt.plot(years, slight_by_year, color='#e91e63')

plt.ylim(0,2000)
plt.xlim(2014,2018)

plt.xlabel('Year', fontsize=18)
plt.ylabel('Number of Fatal Casualties', fontsize=18)

## are young drivers involved in more accidents?

In [ ]:
df_vehicles['Age_Band_of_Driver'].value_counts().sort_index().plot(kind='bar')

# https://www.confused.com/car-insurance/guides/how-car-insurance-is-calculated

## Are there more accidents during night, daytime, sunrise or sunset?

categorise time
- night
- Civil Twilight:
- daylight

## can we find the worst accident hot spot locations?

https://andrewpwheeler.com/2015/09/03/using-kdtrees-in-python-to-calculate-neighbor-counts/

In [ ]:
df_accidents_coords = df_accidents[['Accident_Index','Location_Easting_OSGR','Location_Northing_OSGR']].copy()

print(len(df_accidents_coords[df_accidents_coords.isnull().any(1)]))
df_accidents_coords_complete = df_accidents_coords.dropna().copy()

df_accidents_coords_complete[['Location_Easting_OSGR','Location_Northing_OSGR']] = df_accidents_coords_complete[['Location_Easting_OSGR','Location_Northing_OSGR']].astype('Int32')
accident_array = df_accidents_coords_complete.values

In [ ]:
accident_tree = KDTree(accident_array[:,1:3])
df_accidents_coords_complete.loc[:,'NeighbourNO']=accident_tree.query_radius(accident_array[:,1:3],r=70,count_only=True)
df_accidents_coords_complete.sort_values(by='NeighbourNO', ascending=False).reset_index()[:20]

### Try DBSCAN

In [ ]:
X_std = StandardScaler().fit_transform(accident_array[:,1:3])
X = np.array(accident_array[:,1:3], dtype='int32')

In [ ]:
sns.scatterplot(x=X[:,0], y=X[:,1])
plt.axis('equal')

In [ ]:
# Compute DBSCAN
db = DBSCAN(eps=15, min_samples=40, n_jobs=8).fit(X)
labels = db.labels_
len(np.unique(labels))

In [ ]:
df_accidents_coords_complete['Cluster No'] = labels

In [ ]:
plt.figure(figsize=[25,25])

plt.axis('equal')
plt.grid()

# '#e91e63'
plt.scatter(df_accidents_coords_complete["Location_Easting_OSGR"],
            df_accidents_coords_complete["Location_Northing_OSGR"],
            c='#37474F', s=0.4,
            alpha=0.5, label="Accidents")

# '#37474F'
plt.scatter(df_accidents_coords_complete.loc[df_accidents_coords_complete['Cluster No']>0,"Location_Easting_OSGR"],
            df_accidents_coords_complete.loc[df_accidents_coords_complete['Cluster No']>0,"Location_Northing_OSGR"],
            c=df_accidents_coords_complete.loc[df_accidents_coords_complete['Cluster No']>0,'Cluster No'],
            s=20, label="Cluster of accidents")

plt.title('Accidents')

# plt.ylim(0,200000)
# plt.xlim(100000,350000)

plt.xlabel('OSGR Easting [m]')
plt.ylabel('OSGR Northing [m]')
plt.yticks([50000,100000,150000,200000,250000,300000])
#plt.grid(linestyle=':')
plt.grid(False)
plt.legend(prop={'size': 16})

In [ ]:
df_accidents_coords_complete.loc[df_accidents_coords_complete['Cluster No']==2,:]
df_accidents = df_accidents.merge(df_accidents_coords_complete[['Accident_Index', 'Cluster No']],
                                  left_on='Accident_Index', right_on='Accident_Index', how='left')

In [ ]:
df_accidents.head()
df_accident_clusters = df_accidents[df_accidents['Cluster No']>-1].copy()
len(df_accident_clusters)

In [ ]:
label_to_look_at = 1

In [ ]:
plt.figure(figsize=[25,25])
plt.axis('equal')
plt.grid()

df_accidents_to_look_at = df_accidents[df_accidents['Cluster No']==label_to_look_at].copy()


plt.scatter(df_accidents["Location_Easting_OSGR"],
            df_accidents["Location_Northing_OSGR"],
            c='#37474F', s=0.4,
            alpha=0.5, label="Accidents")

# '#37474F'
plt.scatter(df_accidents_to_look_at["Location_Easting_OSGR"],
            df_accidents_to_look_at["Location_Northing_OSGR"],
            c='#e91e63',
            s=20, label="Cluster of accidents")

plt.title('Accidents')

plt.ylim(df_accidents_to_look_at["Location_Northing_OSGR"].min()-100,df_accidents_to_look_at["Location_Northing_OSGR"].max()+100)
plt.xlim(df_accidents_to_look_at["Location_Easting_OSGR"].min()-100,df_accidents_to_look_at["Location_Easting_OSGR"].max()+100)

plt.xlabel('OSGR Easting [m]')
plt.ylabel('OSGR Northing [m]')
# plt.yticks([50000,100000,150000,200000,250000,300000])
#plt.grid(linestyle=':')
plt.grid(False)
plt.legend(prop={'size': 16})

In [ ]:
fig = px.scatter_mapbox(df_accident_clusters, lat="Latitude", lon="Longitude",
                        hover_name="Cluster No", hover_data=["Accident_Severity", "Number_of_Casualties"],
                        color_discrete_sequence=["fuchsia"], zoom=5, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
df_accident_clusters.head()

## Try Optics

In [ ]:
start_time = time()
optics_clustering = OPTICS(min_samples=40, max_eps=25, metric='euclidean', n_jobs=8).fit(X)

end_time = time()
print((end_time-start_time))

In [ ]:
optics_labels = optics_clustering.labels_
print(len(np.unique(optics_labels)))

df_accidents_coords_complete['OPTICS Cluster No'] = optics_labels

df_accidents = df_accidents.merge(df_accidents_coords_complete[['Accident_Index', 'OPTICS Cluster No']],
                                  left_on='Accident_Index', right_on='Accident_Index', how='left')

In [ ]:
df_accidents.head()

In [ ]:
df_accident_clusters = df_accidents[df_accidents['OPTICS Cluster No_y']>-1].copy()
len(df_accident_clusters)

In [ ]:
fig = px.scatter_mapbox(df_accident_clusters, lat="Latitude", lon="Longitude",
                        hover_name='OPTICS Cluster No_y', hover_data=["Accident_Severity", "Number_of_Casualties"],
                        color_discrete_sequence=["fuchsia"], zoom=5, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Can we estimate the number of casualities and respective severity?

A witness would know:
- Location
- type of Vehicles
- Date/time
- Surface conditions

Out of those what would be relevant?

What are we trying to estimate?
<br>
number of casualities with severity?
get a single estimation with a vector of 3 elements: number of slight, serious, fatal
 - Multivariate Linear Regression
 - ## Let's use ANN

- Veh Leaving Carriageway

<br>get a model for each vehicle?

In [34]:
accident_features = ['Accident_Index','Road_Type','Speed_limit','Junction_Detail','Junction_Control',
                     'Light_Conditions','Weather_Conditions','Road_Surface_Conditions','Urban_or_Rural_Area']

vehicle_features = ['Vehicle_Type']
# Veh Leaving Carriageway

In [26]:
df_vehicle_classes = df_vehicles.groupby(['Accident_Index','Vehicle_Type']).size().unstack(fill_value=0)
print(len(df_vehicle_classes))

675616


In [23]:
casaualty_severity = df_casualties.groupby(['Accident_Index','Casualty_Severity']).size().unstack(fill_value=0)
casaualty_severity.rename(columns={1: "Fatal", 2: "Serious", 3: "Slight"}, inplace=True)
casaualty_severity.head()

Casualty_Severity,Fatal,Serious,Slight
Accident_Index,,,
201401BS70001,0,0,1
201401BS70002,0,0,1
201401BS70003,0,0,1
201401BS70004,0,0,1
201401BS70006,0,0,1


In [27]:
print(len(df_accidents[accident_features]))
df_accidents_s1 = df_accidents[accident_features].merge(df_vehicle_classes, on="Accident_Index")
df_accidents_s2 = df_accidents_s1.merge(casaualty_severity, on="Accident_Index")

675616


In [40]:
# simply drop NA
df_accidents_s3 = df_accidents_s2.drop(df_accidents_s2[df_accidents_s2.isnull().any(1)].index)

In [43]:
df_accidents_s3.columns.to_list()

['Accident_Index',
 'Road_Type',
 'Speed_limit',
 'Junction_Detail',
 'Junction_Control',
 'Light_Conditions',
 'Weather_Conditions',
 'Road_Surface_Conditions',
 'Urban_or_Rural_Area',
 -1,
 1,
 2,
 3,
 4,
 5,
 8,
 9,
 10,
 11,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 90,
 97,
 98,
 'Fatal',
 'Serious',
 'Slight']

In [54]:
x_cont_cols = ['Road_Type', 'Speed_limit', 'Junction_Detail', 'Junction_Control',
 'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 'Urban_or_Rural_Area']

x_cat_cols = [-1, 1, 2, 3, 4, 5, 8, 9, 10, 11, 16, 17, 18, 19, 20, 21, 22, 23, 90, 97, 98]

label_cols = ['Fatal', 'Serious', 'Slight']

In [53]:
X = torch.tensor(df_accidents_s3[x_cols].values, dtype=torch.float)

In [52]:
y = torch.tensor(df_accidents_s3[label_cols].values, dtype=torch.float)

### Setting up Embedding

In [58]:
# category sizes
cat_szs = [len(x_cat_cols)]

In [61]:
# Mebedding sizes
# Half of the cat sizes, max 50
emb_szs = [(size, min(50, (size+1)//2)) for size in cat_szs]
emb_szs

[(21, 11)]

In [64]:
selfembeds = nn.ModuleList([nn.Embedding(ni, nf) for ni,nf in emb_szs])

In [76]:
catz = torch.tensor(df_accidents_s3[x_cat_cols].values,dtype=torch.int)
catz.shape

catz[:,1]

tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)

In [78]:
embedingz = []

for i,e in enumerate(selfembeds):
    embedingz.append(e(catz[:,i]))

In [91]:
embedingz[0].shape
catz[0].shape

torch.Size([21])

In [92]:
e(catz[0]).shape

torch.Size([21, 11])

In [85]:
selfemdrop = nn.Dropout(0.4)

In [86]:
selfemdrop(e(torch.tensor(1)))

tensor([-0.1906, -2.4161,  1.9179, -0.6744, -0.1992, -0.0000, -0.0000,  0.5481,
         0.0000, -0.0000,  2.2873], grad_fn=<MulBackward0>)